# MBID List to recordingsData.json

This code takes a list of MBIDs and creates a json file including the recording list, record informations, pitchSpaces and pitchTracks of each recording.


## 1) Initialization

In [1]:
# Set your token here from https://dunya.compmusic.upf.edu/user/profile/
# This is a random token
token = 'j9fh831536f7f98hp18b3d148klbc2by4k21fx7'

import os 
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from compmusic.dunya import docserver as ds
from compmusic import dunya as dn
from external_utilities.predominantmelodymakam import PredominantMelodyMakam
from external_utilities.pitchdistribution import PitchDistribution
from external_utilities.converter import Converter

from compmusic import dunya
dn.set_token(token)

%matplotlib inline

REF_PITCH=220 

CENTS_IN_OCTAVE= 1200
KOMAS_IN_OCTAVE = 53
KOMA_CENT = CENTS_IN_OCTAVE / KOMAS_IN_OCTAVE

In [2]:
MBID = [ 
        # Hüseyni
        "75a68ff2-7fe0-4012-afbe-f4de3b8a9a7d", # Baygın Suların          0
        #"a8ff9420-e55f-4af7-9089-3fe29686bc54",# Sarı Gelin 
        "cbcf2d1c-8371-4a57-8893-a1958ed001d3", # Demirciler Demir        1
        "f0e215c0-49f8-4962-8263-590dfff02a31",  # Egin Havası            2
        "0eac190d-13c4-442f-bb13-cf734d3cbe88", # Senden Bilirim          3
        # Saba
        "a49bf764-e19a-4c5f-bd74-07450affa35a", # Cümle Hüccac            4
        "d35e07f4-fbf5-4efb-8a36-721565fa8044", # Süzdükçe Güzel          5
        #"e05b2a9f-a129-4847-99ca-562628b97bc1", # Saba Peşrev             
        "a80b0276-f769-433c-8944-d316848409c5", # Bir Esmere Gönül        6
        # Hüzzam
        "e2e5febb-1ad2-4478-adf6-47e4ee534b16", # Yine Kalbim             7
        "220a57b5-3e8a-4a22-93ed-e7400e6844c4", # Şu Göğsüm               8
        "2c2b51b3-150b-4693-9383-9c467cbbdadd", # Sormadın Halimi         9
        # Rast
        "1701ceba-bd5a-477e-b883-5dacac67da43", # Nihansın Dideden Ey    10
        "48fb37f4-0db9-4bf9-8051-51640bbcf6f1", # Bais-i Berbadım        11 
        "26da8cac-5757-4494-a214-25ad564fc292",  # Rast Zeybek           12
        "0321f83e-bee5-4009-983b-1b4fe3c89f23" # Yüzüdür Cihanı          13       
        ]

In [3]:
dataDir = os.path.join('..', 'Oguz', 'data','recordings')
os.makedirs(dataDir, exist_ok=True) 

## 2) Record Information

This part obtains the relevant information about the recording. info = { 'ID' : 'easing', 'Makam', ... }

In [4]:
# enter the easing value corresponding to the recording in the list
easing = [0.3, 0.15, 0.2, 0.2, 0.2, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2 ]
#          0    1    2    3    4    5    6    7    8    9    10    11   12   13   14

The cell below reads the record information from the Info.json file that has the information from musicbrainz

In [7]:
info = dict() # Dictionary for the record information

The cell below is for **reading** information from the **info.json** file.

In [8]:
with open("metaData/Info.json", "r",encoding='utf-8') as readf3:
    info = json.load(readf3)
    
for counter, ID in enumerate(MBID):
    info[ID]['easing'] = easing[counter]

The cell below is for **reading** information from **MusicBrainz**.

for counter, ID in enumerate(MBID):
 
    MB_dict = dn.conn._dunya_query_json("api/makam/recording/%s" %ID) # Retrieve Info From MusicBrainz
    
    info[ID] = {'easing': easing[counter] ,
                'makam': MB_dict['makamlist'][0]['name'],
                'usul': MB_dict['usullist'][0]['name'],
                'title': MB_dict['title'],
                'artist': MB_dict['artists'][0]['name'],
                'link': "https://musicbrainz.org/recording/"+ID,
                'option': '{} ({}, {})'.format(MB_dict['title'], MB_dict['makamlist'][0]['name'],MB_dict['usullist'][0]['name']),
                'trackFile': ID+".mp3"
                #'duration' is put in the Pitch Track part
    }

## 3) Pitch Track and Pitch Series
The cells below: 
1. Creates a pitch track dictionary **pitchTrack = { 'MBID': { 'Hz': pitchTrack, 'cents': pitchTrack } }**, where **pitchTrack = {'time' : 'pitch'} in Hz and cents **,
2. Creates a pitch Series(Hz) dictionary **pitchSeriesHz = {'MBID' : pitch_serie_Hz }**, where **pitch_serie_Hz** is a numpy array and,
3. Creates a pitch track file **MBID.pitch**

'time' incrementation is determined by the  **timeStep**,
 
 $$timeStep = \frac{hopSize}{samplingFrequency}$$
 

In [9]:
hopSize = 441 # for 0.01 seconds timeStep

In [10]:
def compute_pitch(filename):
    
    extractor = PredominantMelodyMakam(hopSize,filter_pitch=True)    
    results = extractor.run(filename)
    
    timeStep = extractor.get_settings()['hopSize'] / extractor.get_settings()['sampleRate']
              
    pitch = results['settings']  # collapse the keys in settings
    pitch['pitch'] = results['pitch']
    pitch['timeStep'] = timeStep
    
    return pitch

In [11]:
pitchTrack = {} # Dictionary for pitch track in Hz and cents
pitchSeriesHz = dict() # Dictionary for the pitch Series in Hz

In [12]:
def truncate(x,y):
    return int(x*(10)**y) / 10**y   

In [13]:
# The dictionary containing the tonic information of all the 70 recordings, taken from subset_annotations.json
with open('metaData/MBID_tonic.json', "r",encoding='utf-8') as readf2:
    tonicDict = json.load(readf2)

 The cell below **creates the pitch file or reads it**
 
 Use smoothed pitch files for accuracy.

In [14]:
for ID in MBID:
    
    mp3_filename = os.path.join(dataDir, '{}.mp3'.format(ID))
    pitchFile = os.path.join('data','pitchTracks','correct_pitchTracks','{}.txt'.format(ID))
    
    tonic_hz = tonicDict[ID] # tonic information is from subset_annotations
   
    pitchTrack[ID] = {'Hz': dict(), 'cents': dict()} 
   
    # If pitch file exists, read it, if not run extractor and create the pitch file
    if not os.path.exists(pitchFile):
        print('Extracting Pitch Track {}'.format(ID))

        pitch = compute_pitch(mp3_filename) # Time / Pitch / Salience 
       
        pitch_serie_Hz = [] # Pitch Track List
        
        with open(pitchFile, 'w') as fp:
            for p_triplet in pitch['pitch']:
                
                # p is for hz, c is for cents
                if not p_triplet[1]:
                    p = 'S'   # put an S for Silent
                    c = 'S'
                else:
                    p = truncate(p_triplet[1],2)
                    # convert it then truncate 
                    c = truncate(Converter.hz_to_cent(p_triplet[1],tonic_hz),2)
                
                # p_triplet is for the .pitch file 
                fp.write(str(p_triplet[0]) + '\t' + str(p_triplet[1] ) + '\n') 
                
                pitch_serie_Hz.append(p_triplet[1]) 
                
                pitchTrack[ID]['Hz'][p_triplet[0]] = p # Add the time key and the pitch to the dict
                pitchTrack[ID]['cents'][p_triplet[0]] = c # Add the time key and the cent to the dict
            
        pitch_serie_Hz = np.array(pitch_serie_Hz)
             
    else:
        print('Loading Pitch Track')
        
        pitchData = np.loadtxt(pitchFile)
        timeStamps = pitchData[:,0]
        pitch_serie_Hz = pitchData[:,1]
        
        for p_duo in pitchData:
            
            if not p_duo[1]:
                p = 'S'
                c = 'S'
            else:
                p = truncate(p_duo[1],2)
                c = truncate(Converter.hz_to_cent(p_duo[1],tonic_hz),2)
            
            pitchTrack[ID]['Hz'][p_duo[0]] = p # Add the time key and the pitch 
            pitchTrack[ID]['cents'][p_duo[0]] = c # Add the time key and the cent 
                            
    pitchSeriesHz[ID] = pitch_serie_Hz # Record the pitch series array in the dictionary
    info[ID]['duration'] = list(pitchTrack[ID]['Hz'].keys())[-1] # Add the duration to the info Key

Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track
Loading Pitch Track


## 4) Pitch Distribution and Pitch Space

This part is for extracting the pitch space of a recording and creating the pitchSpace = { 'MBID' : pitchDict }, where the pitchDict has the pitch and cent information.

In [15]:
# Function definition for automatic scale-interval detection from pitch distribution
def peakLocationDetection(pcd,tresh,ws):
    '''A simple peak detection implementation for demonstration purposes
    Thresholds are manually set for this demo
    '''
    windowSize = ws  # should be odd
    midPointIndex = int(windowSize / 2)
    threshold = np.max(pcd) * tresh
    peakIndexes = []
    for index in range(len(pcd)-windowSize):
        frame = pcd[index:index+windowSize]
        if np.argmax(frame) == midPointIndex and np.max(frame) > threshold:
            peakIndexes.append(index + midPointIndex)
    return peakIndexes

In [16]:
def komaValidator(lst,bound,iterations):
    """
    This function takes a list in cents, a lower bound in cents for identifying two different 
    perdeler, and the number of iterations to be performed to return a new list in cents, that
    are seperated by at least the bound (1 Hc preferably). It keeps the larger of the two perde.    
    """
    assert bound >= 22.0, 'The lower bound must be greater than 22 cents!'
    
    new_lst = lst.copy()
    
    for a in range(iterations):
        
        delta = []

        for i in range(len(new_lst)-1):
            delta.append(new_lst[i+1]-new_lst[i])

        for j in range(len(delta)-1):
            if delta[j] < bound:                
                new_lst.remove( min(new_lst[j],new_lst[j+1])) # discard the smallest
                break # exit the inner for loop
                
    return new_lst        

In [17]:
def shift_to_tonic(lst):
    """
    This function takes a list of a pitch space in cents and returns how much the list is shifted from the tonic,
    with the cents list centered around the tonic.
    """    
    dist = 0
    nlst = lst.copy()

    # if 0.0 is in the list exit code        
    if not (0.0 in nlst):        
        neg = []
        pos = []
            
        for l in nlst:           
            if l < 0:
                neg.append(l)
            elif l> 0:
                pos.append(l)
                        
        if not neg:            
            dist = min(pos)
            nlst = [l - dist  for l in nlst]                        
        else:
            m1 = max(neg)
            m2 = min(pos)

            if abs(m1) > m2:
                dist = m2
                nlst = [l - dist for l in nlst]
            else:
                dist = m1
                nlst = [l - dist for l in nlst]        
        
    return nlst,dist               

### 4.1) Pitch Distribution Calculations

This part computes the **pitch distributions** for each recording using the **peakLocationDetection()** algorithm  and,

creates the **pitchSpace = { 'MBID':  { 'cents': [ ], 'validated cents': [ ], 'tonic shift': { 'cents': [ ], 'distance': float } } }** .

The parameters for pitch distriburion calculation are stored in the **pd_params = {}**

The initial peaks are stored in the **'cents'**, they are koma validated and stored in the **'validated cents'** and finally are shifted so that the tonic is at 0.0 cents in the **'tonic shift[ 'cents' ]**. 

The pitch distribution objects are stored in the **pitchDistrib = { 'ID': predominantMelodymakam() }**. Also all the cent bins are shifted so that 0.0 cent is in the Pitch Space and stored in the **pitchDistrib_binsShifted : {}**

Finally, if you expected to hear a cent between some peaks in the pitch space, you can enter the boundaries of the intervals and append a single cent in that interval to the pitch space in **part 4.2**

In [16]:
pitchSpace = dict() # Dictionary for the Pitch Spaces
pd_params = { MBID[0] : {'step_size': 5,  'kernel_width': 2.5,   'threshold': 0.05,'windowSize':13}, # baygın
             MBID[1] : {'step_size': 5,  'kernel_width': 2.5,   'threshold': 0.04,'windowSize':9}, # demir
             MBID[2] : {'step_size': 5,  'kernel_width': 5,   'threshold': 0.04,'windowSize':13}, # egin
             MBID[3] : {'step_size': 5,  'kernel_width': 2.5,   'threshold': 0.03,'windowSize':11}, #senden
             #"a8ff9420-e55f-4af7-9089-3fe29686bc54" : {'kernel_width': 5, 'step_size': 2.5,'threshold': 0.05,'windowSize':13},
             MBID[4] : {'step_size': 5,  'kernel_width': 2.5,     'threshold': 0.04,'windowSize':13}, #cümle
             MBID[5] : {'step_size': 5,  'kernel_width': 2.5,   'threshold': 0.05,'windowSize':9}, #süzdükçe
             #MBID[6] : {'kernel_width': 5, 'step_size': 2.5,'threshold': 0.05,'windowSize':13}, # saba
             MBID[6] : {'kernel_width': 5, 'step_size': 7.5,'threshold': 0.05,'windowSize':13}, # bir esmere
             MBID[7] : {'step_size': 5,  'kernel_width': 7.5,  'threshold': 0.05,'windowSize':11}, # yine kalbim
             MBID[8] : {'step_size': 5,  'kernel_width': 5,    'threshold': 0.04,'windowSize':11}, # şu göğsüm
             MBID[9] : {'step_size': 5,   'kernel_width': 2.5,    'threshold': 0.04,'windowSize':11}, # sormadın      
             MBID[10] : {'step_size': 2.5, 'kernel_width': 5,    'threshold': 0.03,'windowSize':11}, # nihansın
             MBID[11] : {'step_size': 5, 'kernel_width': 7.5,   'threshold': 0.03,'windowSize':11}, #bais-i berbad
             MBID[12] : {'step_size': 5,   'kernel_width': 7.5,  'threshold': 0.01,'windowSize':13}, #rast zeybek
             MBID[13] : {'step_size': 5,   'kernel_width': 7.5,  'threshold': 0.05,'windowSize':13} #yüzüdür cihan
}

In [17]:
pitchDistrib = {} 

In [18]:
# Creating the Pitch Space
for ID in MBID:
    
    pitchSpace[ID] = {}
    
    # Computing pitch distribution with reference frequency = tonic
    pitch_distribution_tonicRef = PitchDistribution.from_hz_pitch(pitchSeriesHz[ID], tonicDict[ID],\
                                        pd_params[ID]['kernel_width'],pd_params[ID]['step_size'])
    
    # Record the Pitch Distributions    
    pitchDistrib[ID] = pitch_distribution_tonicRef
        
    # PLD peak picking
    peakLocations = list(peakLocationDetection(pitch_distribution_tonicRef.vals,\
                                       pd_params[ID]['threshold'],pd_params[ID]['windowSize']))
   

    # This part is for computing seperate distributions for the lower octave and the higher.
    if ID == MBID[9]:
        pitch_distribution0 = PitchDistribution.from_hz_pitch(pitchSeriesHz[ID], tonicDict[ID],\
                                        5,5)           
        peak_locations0 = list(peakLocationDetection(pitch_distribution0.vals,\
                                       pd_params[ID]['threshold'],pd_params[ID]['windowSize']))
        
        peaks0 = list(pitch_distribution0.bins[peak_locations0])
        
        lower_peaks = [i for i in peaks0 if i < 0.0]
        higher_peaks = [i for i in pitch_distribution_tonicRef.bins[peakLocations] if i>= 0.0 ]
        
        peaks = lower_peaks + higher_peaks
            
    # the peaks in cents
    if not ID == MBID[9]:
        pitchSpace[ID]['cents'] = list(pitch_distribution_tonicRef.bins[peakLocations])
    else:
        pitchSpace[ID]['cents'] = peaks
            
    pitchSpace[ID]['validated cents'] = komaValidator(pitchSpace[ID]['cents'],KOMA_CENT,20) # koma validate the peaks
    
    # center the distribution around the calculated tonic
    shifted,distance = shift_to_tonic(pitchSpace[ID]['validated cents'])
    pitchSpace[ID]['tonic shift'] = {'cents': shifted, 'distance': distance }       

In [19]:
pitchDistrib_binsShifted = {} # all the bins are shifted 
for ID in MBID:
        
    pitchDistrib_binsShifted[ID] = [i-pitchSpace[ID]['tonic shift']['distance'] for i in pitchDistrib[ID].bins]    

### 4.2) Pitch Space Correction

This part is for pitch space correction.You can remove and add certains manually or you can specify an interval in cents and add the most prominent cent in that interval automatically.

**A)** This cell is for **manually** correcting the Pitch Space with a posteori inspection.

In [20]:
# Senden bilirim
pitchSpace[MBID[3]]['tonic shift']['cents'].remove(-905.0)

# Süzdükçe Güzel
pitchSpace[MBID[5]]['tonic shift']['cents'].remove(500.0) # Transition
pitchSpace[MBID[5]]['tonic shift']['cents'].remove(225.0)

# Yine Kalbim
pitchSpace[MBID[7]]['tonic shift']['cents'].remove(565.0) # FM False 

# Nihansın Dideden
pitchSpace[MBID[10]]['tonic shift']['cents'].remove(805.0) # FM False 

# Bais-i Berbadım
pitchSpace[MBID[11]]['tonic shift']['cents'].remove(820.0)
pitchSpace[MBID[11]]['tonic shift']['cents'].remove(1000.0)
pitchSpace[MBID[11]]['tonic shift']['cents'].remove(1080.0)
pitchSpace[MBID[11]]['tonic shift']['cents'].remove(1130.0)

# Rast Zeybek
pitchSpace[MBID[12]]['tonic shift']['cents'].remove(605.0) 

**B)** This part is for specifying the intervals that you expect a peak and **automatically updating** the pitch space

Enter the **boundaries** of the frequency intervals where you expect a peak and leave the list **empty otherwise**

**Note**: Be sure that the boundary values are **increasing!**

In [21]:
# MBID[2]: [495, 700, 1005, 1190]    MBID[4]: [-100, -50, 565, 685, 705, 810, 1050, 1100]
boundaries = {MBID[0]: [], MBID[1]: [] , MBID[2]: [], MBID[3]: [], 
              MBID[4]: [], MBID[5]: [500.0,600.0,745.0,800.0], MBID[6]: [],
              MBID[7]: [], MBID[8]: [], MBID[9]: [], MBID[10]: [],
              MBID[11]: [], MBID[12]: [], MBID[13]: []} 

new_cents = {MBID[0]: [], MBID[1]: [], MBID[2]: [], MBID[3]: [], MBID[4]: [], MBID[5]: [], MBID[6]: [], MBID[7]: [], MBID[8]: [], 
             MBID[9]: [], MBID[10]: [],MBID[11]: [], MBID[12]: [],MBID[13]: []}

In [22]:
for ID in MBID: 

    for i in range(int(len(boundaries[ID])/2)):
        
        # find the interval
        b0 = boundaries[ID][2*i]
        b1 = boundaries[ID][2*i+1]

        #find the indexes of the boundaries
        i0 = pitchDistrib_binsShifted[ID].index(b0)
        i1 = pitchDistrib_binsShifted[ID].index(b1)

        interval = range(i0,i1)

        # the values in the interval
        A = pitchDistrib[ID].vals[interval]

        # use peak detection on the interval
        small_peak_locs = list(peakLocationDetection(A, 0.05 ,3))

        # find the index inside the interval
        ind = np.where(A == max(A[small_peak_locs]))[0][0]

        # find the corresponding cent
        cent = pitchDistrib_binsShifted[ID][i0+ind]
        new_cents[ID].append(cent)

    for c in new_cents[ID]:
        pitchSpace[ID]['tonic shift']['cents'].append(c)
                
    pitchSpace[ID]['tonic shift']['cents'].sort()

### 4.3) Perde Dictionary

This part is for creating the perde dictionary including perde names, perde functions and keyboard mappings.

Load the **Makam Dictionary(MakamInformation.json)** including the perde names and tonal information of the makams.
The information here is taken from the Turkish Makam Music Guide, Murat Aydemir, 2010, İstanbul.

In [23]:
with open('metaData/MakamInformation.json', "r",encoding='utf-8') as rf:
    makamDict = json.load(rf)

In [24]:
def perdeNameFinder(cent,makamName,makamDict,eps):
    """
    This function takes a cent value, the name of the Makam, the Makam Dictionary,
    and an epsilon value for interval searching and returns the name of the perde.
    Perde names are taken from the AEU system.
    """
    makam = makamDict[makamName]
    name =''    
    
    for i in range(len(makam['cents'])):
        if abs(cent-makam['cents'][i]) <= eps:
            name = makam['perdeler'][i]

    return name

In [25]:
def functionFinder(perdeName,makamName):
    """
    This method takes the name of a perde and the Makam of interest and returns the function of the perde. 
    """
    function = ''

    if makamName == 'Rast' and (perdeName == 'Rast' or perdeName == 'Gerdaniye' ):
        function = 'tonic'
    elif makamName == 'Rast' and perdeName == 'Neva':
        function = 'dominant'

    if makamName == 'Hüzzam' and (perdeName == 'Segah' or perdeName == 'Tiz Segah'):
        function = 'tonic'
    elif makamName == 'Hüzzam' and perdeName == 'Neva':
        function = 'dominant'

    if makamName == 'Hüseyni' and (perdeName == 'Dügah' or perdeName == 'Muhayyer'):
        function = 'tonic'
    elif makamName == 'Hüseyni' and perdeName == 'Hüseyni':
        function = 'dominant'

    if makamName == 'Saba' and (perdeName == 'Dügah'):
        function = 'tonic'
    elif makamName == 'Saba' and perdeName == 'Çargah':
        function = 'dominant'

    return function    

The cell below creates a list of pitch dictionaries and forms the **Pitch Space**.

In [26]:
keyboard = {'lower octave': ['z','x','c','v','b','n','m','ö','ç'], 
            'main octave': ['a','s','d','f','g','h','j','k','l','ş','i'],
            'higher octave': ['q','w','e','r','t','y','u','ı','o','p']}

In [27]:
for ID in MBID:
    
    pitchSpace[ID]['pitch space'] = []
    
    makamName = info[ID]['makam'] # Retrieve the makam name
    makam = makamDict[makamName]  # Accessing the tonal information dictionary
    
    perdeler = pitchSpace[ID]['tonic shift']['cents']
      
    # Keyboard button mapping needs accurate pitch spaces    
    counter0 = -1;counter1 = -1;counter2 = -1

    for perde in perdeler:

        perdeName = perdeNameFinder(perde,makamName,makamDict,KOMA_CENT)
        
        button = ""
                
        if perde < 0.0:
            counter0 += 1

            if counter0 <= len(keyboard['lower octave'])-1:
                button = keyboard['lower octave'][counter0]

        elif perde >= 0.0 and perde < 1200-KOMA_CENT:
            counter1 += 1

            if counter1 <= len(keyboard['main octave'])-1:
                button = keyboard['main octave'][counter1]    

        elif perde >= 1200-KOMA_CENT:
            counter2 += 1

            if counter2 <= len(keyboard['higher octave'])-1:
                button = keyboard['higher octave'][counter2]


        pitchDict = {"perdeName": perdeName,  
             "pitch": truncate(Converter.cent_to_hz(perde,tonicDict[ID]),2),# cents to hz, truncate after 2
             "cent": perde,
             "key" : button,
             "function": functionFinder(perdeName,makamName)
        }

        pitchSpace[ID]['pitch space'].append(pitchDict)

## 5) Melody Dictionary

This part combines the pitchTrack and the pitchSpace dictionaries into a single melody dictionary: **melody = { 'MBID' : pitchSpace, pitchTrack }**

In [28]:
melody = dict()

In [29]:
for ID in MBID:
        
    melody[ID] = {'pitchSpace': pitchSpace[ID]['pitch space'],
                  'pitchTrack': pitchTrack[ID]['cents']                          
    }        

## 6) Record the Data

dataDict is a dictionary containing: 
1. **'recordingsList' = [  ]**,  a list of all the MBIDs,
2. **{ 'MBID' : info, melody }**: a dictionary containing:     
.**{ 'info': ...  }** the record information dictionary,    
.**{ 'melody' : pitchSpace, pitchTrack}** a dictionary containing the pitchSpace list and pitchTrack dictionary.

In [30]:
dataDict = dict() # Dictionary Containing all the data

In [31]:
dataDict['recordingsList'] = MBID

for ID in MBID:

    dataDict[ID] = {'info': info[ID],
                    'melody': melody[ID]        
    }        

## 7) recordingsData.json

In [32]:
with open("recordingsInfo.json", "w",encoding='utf-8') as write_file:
    json.dump(dataDict, write_file, indent=0,ensure_ascii=False)